# Housing Market Regression Challenge
The housing market is one of the most crucial parts of the economy for every country. Purchasing a home is one of the primary ways to build wealth and savings for people. In this respect, predicting prices in the housing market is a very central topic in economic and financial circles.

The house price dataset from Kaggle includes several features of the houses along with their sale prices at the time they are sold. So far, in this module, you built and implemented some models using this dataset.

In this challenge, you are required to improve your model with respect to its prediction performance.

To complete this challenge, submit a Jupyter notebook containing your solutions to the following tasks.

Steps
1. Load the houseprices data from Thinkful's database.
2. Do data cleaning, exploratory data analysis, and feature engineering. You can use your previous work in this module. But make sure that your work is satisfactory.
3. Now, split your data into train and test sets where 20% of the data resides in the test set.
4. Build several linear regression models including Lasso, Ridge, or ElasticNet and train them in the training set. Use k-fold cross-validation to select the best hyperparameters if your models include one!
5. Evaluate your best model on the test set.
6. So far, you have only used the features in the dataset. However, house prices can be affected by many factors like economic activity and the interest rates at the time they are sold. So, try to find some useful factors that are not included in the dataset. Integrate these factors into your model and assess the prediction performance of your model. Discuss the implications of adding these external variables into your model.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
from scipy.stats.mstats import winsorize
from sqlalchemy import create_engine
%matplotlib inline
from sklearn import linear_model
import statsmodels.api as sm

from scipy.stats import bartlett
from scipy.stats import levene
from statsmodels.tsa.stattools import acf
from scipy.stats import jarque_bera
from scipy.stats import normaltest
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from statsmodels.tools.eval_measures import mse, rmse

from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV

import warnings
warnings.filterwarnings('ignore')

### 1. Load the houseprices data from Thinkful's database.

In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

main_df = pd.read_sql_query('select * from houseprices',con=engine)

# Close the connection after the query
engine.dispose()

# Take a look
main_df.head(8)

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,None,IR1,Lvl,AllPub,...,0,None,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,None,IR1,Lvl,AllPub,...,0,None,None,Shed,350,11,2009,WD,Normal,200000


### 2. Do data cleaning, exploratory data analysis, and feature engineering.

#### First we'll clean the data.

Let's start by creating a copy of the original data to clean without messing with the raw data. Then we can find all the columns that are missing data, and how many values each is missing.

In [3]:
# Copy the dataframe
haus_df = main_df.copy()

# Count the number of n/a values and print them out
missing_list = haus_df.isna().sum()

print(missing_list[missing_list > 0])

lotfrontage      259
alley           1369
masvnrtype         8
masvnrarea         8
bsmtqual          37
bsmtcond          37
bsmtexposure      38
bsmtfintype1      37
bsmtfintype2      38
electrical         1
fireplacequ      690
garagetype        81
garageyrblt       81
garagefinish      81
garagequal        81
garagecond        81
poolqc          1453
fence           1179
miscfeature     1406
dtype: int64


Looks like the garage variables have a uniform number of missing values. Most likely when houses didn't have garages, these areas just didn't get filled in, so I think it's appropriate to label these as "No Garage". Similarly with the veneer variables.

In [4]:
garages = ['garagetype', 'garagefinish', 'garagequal', 'garagecond']

# Fill in empty garage string values with "No Garage"
for garage in garages:
    haus_df[garage].fillna('NoGarage', inplace=True)

# Use 0 for the year the garage was built instead of a string
haus_df['garageyrblt'].fillna(0.0, inplace=True)

# Also fill in veneer variables
haus_df['masvnrtype'].fillna('NoVeneer', inplace=True)

# Area is a number so we mark it zero
haus_df['masvnrarea'].fillna(0.0, inplace=True)

Since the data is telling us the linear feet of street connected to properties, we can likely fill in the "Lot Frontage" variable with an average of what is seen by neighborhood, so let's go ahead with that.

In [5]:
# Group by the neighborhood and fill in the average of the neighborhood's lot frontage
haus_df['lotfrontage'] = haus_df.groupby(['neighborhood'], sort=False)['lotfrontage'].apply(lambda x: x.fillna(x.mean()))

Seeing as how there are some missing basement variables, but the "TotalBsmtSF" (Total square feet of basement area) variable has no missing values, we can say that any time this variable equals zero, the other basement values are either none or zero (where relevant).

In [6]:
bsmt_list = ['bsmtqual', 'bsmtcond', 'bsmtexposure', 'bsmtfintype1', 'bsmtfintype2']

for bsmt in bsmt_list:
    haus_df.loc[haus_df.totalbsmtsf == 0, bsmt] = 'NoBasement'

This fixed almost all of the null values in the basement variables, however there are still 2 null values to look at (one in bsmtexposure and the other in bsmtfintype2). Let's print these out with relevant basement values.

In [7]:
haus_df.loc[haus_df.bsmtexposure.isna() == True, ['totalbsmtsf', 'bsmtqual', 'bsmtcond',
                                                  'bsmtexposure', 'bsmtfintype1', 'bsmtfintype2']]

,totalbsmtsf,bsmtqual,bsmtcond,bsmtexposure,bsmtfintype1,bsmtfintype2
948,936,Gd,TA,None,Unf,Unf


In [8]:
haus_df.loc[haus_df.bsmtfintype2.isna() == True, ['totalbsmtsf', 'bsmtqual', 'bsmtcond',
                                                  'bsmtexposure', 'bsmtfintype1', 'bsmtfintype2']]

,totalbsmtsf,bsmtqual,bsmtcond,bsmtexposure,bsmtfintype1,bsmtfintype2
333,3206,Gd,TA,No,GLQ,None


Clearly both of these houses have basements, but maybe those values were just ignored in the data entry process. Let's assume that there is no exposure in the first example (948) and assign it as "No", and also assume that there was not a second finish type in the second example (333), or "Unf".

In [9]:
haus_df.loc[haus_df.bsmtexposure.isna() == True, 'bsmtexposure'] = "No"

haus_df.loc[haus_df.bsmtfintype2.isna() == True, 'bsmtfintype2'] = "Unf"

Moving on, if there aren't any fireplaces, obviously we can't measure their quality. Let's fill in any values that match this criteria.

In [10]:
haus_df.loc[haus_df.fireplaces == 0, 'fireplacequ'] = 'NoFireplace'

Similar to the above issues, if the pool's area is 0, then clearly it doesn't exist and we can't quantify its quality.

In [11]:
haus_df.loc[haus_df.poolarea == 0, 'poolqc'] = 'NoPool'

One other oddity is that there is one house that has a null value for electricity, yet has central air which is run by electricity. So let's assume that this value can be filled in by the most common type of electricity that houses with central air has, which I have shown below is standard circuit breakers.

In [12]:
haus_df.loc[haus_df.centralair == "Y", 'electrical'].describe()

count      1364
unique        4
top       SBrkr
freq       1282
Name: electrical, dtype: object

In [13]:
haus_df.loc[haus_df.electrical.isnull(), :] = "SBrkr"

For the last few empty values for "alley", "fence" and "miscfeature", I will simply replace these with more descriptive values, as the null value is expected in the data description as a lack of these variables.

In [14]:
haus_df.loc[haus_df.alley.isnull(), 'alley'] = 'NoAlley'

haus_df.loc[haus_df.fence.isnull(), 'fence'] = 'NoFence'

haus_df.loc[haus_df.miscfeature.isnull(), 'miscfeature'] = 'NoMiscFeatures'

There is also one row where every value is labeled as 'SBrkr', so let's get rid of that.

In [15]:
haus_df = haus_df[haus_df['mssubclass'] != 'SBrkr']

#### Exploratory data analysis and feature engineering:

First let's make a copy of the house data so we can convert our categorical values to numeric values and find correlations.

In [16]:
# Make a copy of the dataframe
corr_haus = haus_df.copy()

# Create a list of columns that aren't already number values so we can give them dummy variables
str_corr_list = ['mszoning', 'street', 'alley', 'lotshape', 'landcontour', 'utilities', 'lotconfig',
       'landslope', 'neighborhood', 'condition1', 'condition2', 'bldgtype',
       'housestyle', 'roofstyle', 'roofmatl', 'exterior1st', 'exterior2nd', 'masvnrtype',
       'exterqual', 'extercond', 'foundation', 'bsmtqual', 'bsmtcond', 'bsmtexposure', 'bsmtfintype1',
       'bsmtfintype2', 'heating', 'heatingqc', 'centralair', 'electrical', 'kitchenqual',
       'functional', 'fireplacequ', 'garagetype', 'garagefinish', 'garagequal',
       'garagecond', 'paveddrive', 'poolqc', 'fence', 'miscfeature', 'saletype', 'salecondition']

# Get dummy variables for each category, and concatenate them back into the dataframe
for col in str_corr_list:
    corr_haus = pd.concat([corr_haus, pd.get_dummies(corr_haus[col])], axis=1)

# Then remove the columns with these strings so we can calculate correlations
for col in str_corr_list:
    corr_haus = corr_haus.drop(col, axis=1)

The other columns not listed above are all object variables however, so we have to update these to be integers so we can calculate correlations

In [17]:
# Create a full list again
tot_list = haus_df.columns.tolist()

# Now we can just take the elements out that we didn't use last time
int_corr_list = [x for x in tot_list if x not in str_corr_list]

# iterate through this new list and make the columns numeric
for icol in int_corr_list:
    corr_haus[icol] = pd.to_numeric(corr_haus[icol])

Now that everything is a number, let's calculate the correlations.

In [18]:
# Calculate the correlation matrix
corr_mtrx = corr_haus.corr()
corr_mtrx = pd.DataFrame(data=corr_mtrx)

# We're just interested in relations to salesprice so we can now limit the correlations
# to just this column
sale_corr = corr_mtrx['saleprice']

# Now get the top 10 positive correlations and take a look at them
best_sale = sale_corr.sort_values(ascending=False).head(11)
best_sale = best_sale.drop('saleprice')

print(best_sale)

overallqual     0.791069
grlivarea       0.708618
garagecars      0.640473
garagearea      0.623423
totalbsmtsf     0.613905
firstflrsf      0.605968
fullbath        0.560881
Ex              0.553093
totrmsabvgrd    0.533779
yearbuilt       0.523273
Name: saleprice, dtype: float64


In words, these variables are (in order):
1. OverallQual: Overall material and finish quality
2. GrLivArea: Above grade (ground) living area square feet
3. GarageCars: Size of garage in car capacity
4. GarageArea: Size of garage in square feet
5. TotalBsmtSF: Total square feet of basement area
6. 1stFlrSF: First Floor square feet
7. FullBath: Full bathrooms above grade (ground)
8. Ex: this is an entry in the "overallqual" column that is being picked up on its own from the dummy variables, so it can be left out
9. TotRmsAbvGrd: Total rooms above ground (does not include bathrooms)
10. YearBuilt: Original construction date

For this exercise, I'll focus on the following features:
1. OverallQual: Overall material and finish quality
2. GrLivArea: Above grade (ground) living area square feet
3. GarageCars: Size of garage in car capacity
4. TotalBsmtSF: Total square feet of basement area

In [19]:
# Create a new dataframe with just the features we want
feat_haus = haus_df[['saleprice', 'overallqual','grlivarea', 'garagecars', 'totalbsmtsf']]

# Get dummy variables for the overall quality column, and concatenate them back into the dataframe
feat_haus = pd.concat([feat_haus, pd.get_dummies(feat_haus['overallqual'])], axis=1)

# Then remove the columns with these strings so we can calculate correlations
feat_haus = feat_haus.drop('overallqual', axis=1)

In [20]:
# Make sure to convert object variable columns to numbers
int_list = ['saleprice', 'grlivarea', 'garagecars', 'totalbsmtsf']

# iterate through this new list and make the columns numeric
for icol in int_list:
    feat_haus[icol] = pd.to_numeric(feat_haus[icol])

Now we can run the house prices model and assess the goodness of fit using F-test, R-squared, adjusted R-squared, AIC and BIC.

In [21]:
# Y is the target variable
Y = feat_haus['saleprice']

# X is the feature set
X = feat_haus[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10,'grlivarea', 'garagecars', 'totalbsmtsf']]

# Create a LinearRegression model object from scikit-learn's linear_model module.
lrm = linear_model.LinearRegression()

# Fit the OLS model using statsmodels module
results = sm.OLS(Y, X).fit()

# Then print a summary report
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              saleprice   R-squared:                       0.794
Model:                            OLS   Adj. R-squared:                  0.792
Method:                 Least Squares   F-statistic:                     463.8
Date:                Sun, 04 Aug 2019   Prob (F-statistic):               0.00
Time:                        15:45:50   Log-Likelihood:                -17380.
No. Observations:                1459   AIC:                         3.479e+04
Df Residuals:                    1446   BIC:                         3.485e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
1            1.431e+04   2.57e+04      0.557      

This model's F-statistic gives us an associated p-value of nearly zero, so this tells us that our chosen features are useful in explaining the the target variable, which in this case is the sale price. The R-squared value is 0.794 and the Adj R-Squared score is 0.792, which means that we are able to explain a large percentage of our variance with this model. However, this could probably perform somewhat better as it still leaves out ~20.8% of the variance explanation (i.e. 1-0.792). Lastly, the AIC and BIC scores are 0.0003479 and 0.0003485 respectively, which are very small values, and thus indicates this model has good explanation power.

This model is mostly satisfactory but it could be improved. Notice that some of the features are not statistically significant, i.e. when the overall quality of the house gets a rating of 1, 2, or 3. I believe if we got rid of features that are not statistically significant we could improve on our model.

In [22]:
# set X as the new feature set
X = feat_haus[[4, 5, 6, 7, 8, 9, 10,'grlivarea', 'garagecars', 'totalbsmtsf']]

# Recreate the LinearRegression model object
lrm = linear_model.LinearRegression()

# Fit the OLS model
results = sm.OLS(Y, X).fit()

# Then print a summary report
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:              saleprice   R-squared (uncentered):                   0.967
Model:                            OLS   Adj. R-squared (uncentered):              0.966
Method:                 Least Squares   F-statistic:                              4201.
Date:                Sun, 04 Aug 2019   Prob (F-statistic):                        0.00
Time:                        15:45:50   Log-Likelihood:                         -17380.
No. Observations:                1459   AIC:                                  3.478e+04
Df Residuals:                    1449   BIC:                                  3.483e+04
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------

This second model is an improvement and I am more comfortable moving forward with it now. The R-Squared and Adj R-Squared scores are higher, the p-value produced by the F-Statistics is still very close to zero, and the AIC and BIC scores even dropped a tiny bit. On top of this all of our variables are statistically significant in this model, unlike the previous one.

### 3. Split your data into train and test sets where 20% of the data resides in the test set.

In [23]:
# Split them up into X/Y Train/Test portions, giving 20% to the test size
# Also give the random state a seed # so we can get the same results each run
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

### 4. Build several linear regression models including Lasso, Ridge, or ElasticNet and train them in the training set. Use k-fold cross-validation to select the best hyperparameters if your models include one!

First I'll write a function to print out all of the relevant statistics that I'll want to look at so I don't have to re-write it every time.

In [24]:
def model_stats():
    print("R-squared of the model in the training set is: {}".format(lrm.score(x_train, y_train)))
    print("-----Test set statistics-----")
    print("R-squared of the model in the test set is: {}".format(lrm.score(x_test, y_test)))
    print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
    print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
    print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
    print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

#### OLS

In [25]:
# Fit an OLS model
lrm = LinearRegression()
lrm.fit(x_train, y_train)

# Make predictions
y_preds_train = lrm.predict(x_train)
y_preds_test = lrm.predict(x_test)

# Print out important stats
model_stats()

R-squared of the model in the training set is: 0.7770214565819715
-----Test set statistics-----
R-squared of the model in the test set is: 0.8707494734498953
Mean absolute error of the prediction is: 20418.828836253237
Mean squared error of the prediction is: 718500399.1862898
Root mean squared error of the prediction is: 26804.857753517175
Mean absolute percentage error of the prediction is: 13.290880223412364


#### Lasso

In [26]:
# Create a list of alpha values to use in the Lasso, Ridge, and ElasticNet models
alphas = [np.power(10.0,p) for p in np.arange(-10,40,1)]

# Fit a Lasso model
lasso_cv = LassoCV(alphas=alphas, cv=3)

lasso_cv.fit(x_train, y_train)

# Make predictions
y_preds_train = lasso_cv.predict(x_train)
y_preds_test = lasso_cv.predict(x_test)

# Print out important stats
print("Best alpha value is: {}".format(lasso_cv.alpha_))
model_stats()

Best alpha value is: 1e-10
R-squared of the model in the training set is: 0.7770214565819715
-----Test set statistics-----
R-squared of the model in the test set is: 0.8707494734498953
Mean absolute error of the prediction is: 20418.828836251916
Mean squared error of the prediction is: 718500399.1862442
Root mean squared error of the prediction is: 26804.857753516324
Mean absolute percentage error of the prediction is: 13.290880223410825


#### Ridge

In [27]:
# Fit a Ridge model
ridge_cv = RidgeCV(alphas=alphas, cv=3)

ridge_cv.fit(x_train, y_train)

# Make predictions
y_preds_train = ridge_cv.predict(x_train)
y_preds_test = ridge_cv.predict(x_test)

# Print out important stats
print("Best alpha value is: {}".format(ridge_cv.alpha_))
model_stats()

Best alpha value is: 1e-10
R-squared of the model in the training set is: 0.7770214565819715
-----Test set statistics-----
R-squared of the model in the test set is: 0.8707494734498953
Mean absolute error of the prediction is: 20418.828836186596
Mean squared error of the prediction is: 718500399.1837006
Root mean squared error of the prediction is: 26804.857753468877
Mean absolute percentage error of the prediction is: 13.290880223328047


#### ElasticNet

In [28]:
# Fit an ElasticNet model
elasticnet_cv = ElasticNetCV(alphas=alphas, cv=3)

elasticnet_cv.fit(x_train, y_train)

# Make predictions
y_preds_train = elasticnet_cv.predict(x_train)
y_preds_test = elasticnet_cv.predict(x_test)

# Print out important stats
print("Best alpha value is: {}".format(elasticnet_cv.alpha_))
model_stats()

Best alpha value is: 1e-10
R-squared of the model in the training set is: 0.7770214565819715
-----Test set statistics-----
R-squared of the model in the test set is: 0.8707494734498953
Mean absolute error of the prediction is: 20418.828797590108
Mean squared error of the prediction is: 718500397.6819184
Root mean squared error of the prediction is: 26804.85772545563
Mean absolute percentage error of the prediction is: 13.290880174407235


### 5. Evaluate your best model on the test set.

### 6. Try to find some useful factors that are not included in the dataset. Integrate these factors into your model and assess the prediction performance of your model. Discuss the implications of adding these external variables into your model.